<a href="https://colab.research.google.com/github/mrmamon/GPT-Fine-Tuning/blob/main/GPT_3_5_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q datasets openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.2 MB/s eta 0:00:00


In [4]:
import openai
from datasets import load_dataset
import json

In [5]:
dataset_name = "FourthBrainGenAI/MarketMail-AI"
product_name = "product"
product_desc = "description"
product_ad = "marketing_email"

In [6]:
dataset = load_dataset(dataset_name)
print(dataset)
print(dataset['train'][0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/17 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['product', 'description', 'marketing_email'],
        num_rows: 17
    })
})
{'product': 'SmartEyes', 'description': 'Glasses with real-time translation', 'marketing_email': "Subject Line: 👓 Unleash Multilingual Magic with SmartEyes Glasses - See the World in YOUR Language! 🌍\n\nHey there, Language Lover! 🌟\n\nDo you dream of seamlessly navigating through conversations in any language? 🙌 Guess what! Your linguistic aspirations are about to come true with our *brand new* product - SmartEyes Glasses! 😎\n\nPicture this - you're strolling through the bustling streets of Tokyo 🇯🇵, admiring the stunning sakura blossoms. Suddenly, a friendly local strikes up a conversation. Worry not! Your SmartEyes Glasses are here to help! 🌸✨\n\nWhy choose SmartEyes, you ask? Here's the scoop: \n\n🎯 Real-time Translation: With the cutting-edge technology packed into SmartEyes Glasses, experience lightning-fast translations, making exchanges smooth and na

In [10]:
dataset['train']

Dataset({
    features: ['product', 'description', 'marketing_email'],
    num_rows: 17
})

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['product', 'description', 'marketing_email'],
        num_rows: 17
    })
})

In [19]:
def write_dataset_to_file(dataset: dict, filename: str) -> None:
    with open(filename, 'w') as f:
        for item in dataset['train']:
            product = item['product']
            description = item['description']
            marketing_email = item['marketing_email']

            formatted_data = {
                "messages": [
                    {
                        "role": "system",
                        "content": "Below is a product and description, please write a marketing email for this product"
                    },
                    {
                        "role": "user",
                        "content": f"### Product:\n{product}\n### Description:\n{description}"
                    },
                    {
                        "role": "assistant",
                        "content": f"{marketing_email}"
                    }
                ]
            }

            f.write(json.dumps(formatted_data) + "\n")

In [20]:
# Write the formatted dataset to a text file
write_dataset_to_file(dataset, "output.txt")

------


In [21]:
openai.api_key = 'sk-ABC'

In [22]:
# Upload files
openai.File.create(
  file=open("/content/output.txt", "rb"),
  purpose='fine-tune'
)

<File file id=file-YdZDm1WccUebBNA0UHnwtcpO at 0x7e30755a9fd0> JSON: {
  "object": "file",
  "id": "file-YdZDm1WccUebBNA0UHnwtcpO",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 38045,
  "created_at": 1693113316,
  "status": "uploaded",
  "status_details": null
}

In [23]:
# Create a fine-tuning job
import os
openai.FineTuningJob.create(training_file="file-YdZDm1WccUebBNA0UHnwtcpO", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-j8EBd56fvgjQYfxq8QZgIGXl at 0x7e307d654c70> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-j8EBd56fvgjQYfxq8QZgIGXl",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693113340,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-4sANCWnrXSjEpD3dzyzqxaIY",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-YdZDm1WccUebBNA0UHnwtcpO",
  "hyperparameters": {
    "n_epochs": 5
  },
  "trained_tokens": null
}

In [24]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7e306d725170> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-j8EBd56fvgjQYfxq8QZgIGXl",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693113340,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-4sANCWnrXSjEpD3dzyzqxaIY",
      "result_files": [],
      "status": "running",
      "validation_file": null,
      "training_file": "file-YdZDm1WccUebBNA0UHnwtcpO",
      "hyperparameters": {
        "n_epochs": 5
      },
      "trained_tokens": null
    }
  ],
  "has_more": false
}

In [27]:
# List up to 10 events from a fine-tuning job
openai.FineTuningJob.list_events(id="ftjob-j8EBd56fvgjQYfxq8QZgIGXl", limit=10)

<OpenAIObject list at 0x7e307d641d00> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-AcVI1Sb9f1j9P7cF11ooEZis",
      "created_at": 1693113902,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-lLK5W1QZQaqInTLtP1vMyaZN",
      "created_at": 1693113900,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::7s2YdAGC",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-6pvk4nos7agVZvxL9LtQQeZe",
      "created_at": 1693113895,
      "level": "info",
      "message": "Step 85/85: training loss=1.06",
      "data": {
        "step": 85,
        "train_loss": 1.0633130073547363,
        "train_mean_token_accuracy": 0.6900269389152527
      },
      "type": "metrics"
  

In [33]:
completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::7s2YdAGC",
  messages=[
    {"role": "system", "content": "Below is a product and description, please write a marketing email for this product"},
    {"role": "user", "content": f"### Product:\nThe Coolinator\n### Description:\nA personal cooling device to keep you from getting overheated on a hot summer's day!"}
  ]
)
ret = completion.choices[0].message
print(ret)

{
  "role": "assistant",
  "content": "Subject: \ud83c\udf2c\ufe0fStay Chill with The Coolinator! \ud83c\udf1e\n\nHey there, \ud83c\udf1f\n \nAre you tired of feeling like you're melting under the summer sun? \u2600\ufe0f Look no further because The Coolinator is here to rescue you from the clutches of scorching heat! \ud83d\ude92\n\nIntroducing The Coolinator - Your Ultimate Personal Cooling Device! \ud83e\udd29\n\nImagine this: You're lounging at the beach \ud83c\udfd6\ufe0f or maybe even out for a run \ud83c\udfc3 . Suddenly, the heatwave hits! But rest easy because you've got The Coolinator strapped right by your side! \ud83d\udd04\n\n\ud83d\udca8 With its advanced fan technology, The Coolinator is engineered to keep you breezy, ensuring maximum comfort in any situation. \n\n\u2705 Main Features:\n\u2705 Compact, lightweight, and portable\n\u2705 Powerful fan with adjustable speed options\n\u2705 USB rechargeable for ultimate convenience\n\u2705 Sleek and trendy design\n\u2705 Long

In [36]:
from IPython.display import display, Markdown
display(Markdown(ret['content']))

Subject: 🌬️Stay Chill with The Coolinator! 🌞

Hey there, 🌟
 
Are you tired of feeling like you're melting under the summer sun? ☀️ Look no further because The Coolinator is here to rescue you from the clutches of scorching heat! 🚒

Introducing The Coolinator - Your Ultimate Personal Cooling Device! 🤩

Imagine this: You're lounging at the beach 🏖️ or maybe even out for a run 🏃 . Suddenly, the heatwave hits! But rest easy because you've got The Coolinator strapped right by your side! 🔄

💨 With its advanced fan technology, The Coolinator is engineered to keep you breezy, ensuring maximum comfort in any situation. 

✅ Main Features:
✅ Compact, lightweight, and portable
✅ Powerful fan with adjustable speed options
✅ USB rechargeable for ultimate convenience
✅ Sleek and trendy design
✅ Long-lasting battery life

Ready to take your summer game to a whole new level? 💯

For a limited time only, shop now and get an exclusive 15% discount using the code CHILL15. You wouldn't want to miss out on this cool 🔥 offer!

Hurry and click the link below to put yourself in full control of your summer destiny! 💪

🛒 Start Chillin' with The Coolinator 🛒

Stay cool, stay fresh, and remember, chill vibes only! ✌️

Wishing you endless coolness,
Team The Coolinator 🌬️